# Module 03: Moving Averages and Trend Analysis

**Difficulty**: ⭐⭐ (Beginner-Intermediate)

**Estimated Time**: 75 minutes

**Prerequisites**: 
- Completed Module 00: Setup and Introduction
- Completed Module 01: Bursa Malaysia Fundamentals
- Completed Module 02: Data Collection with yfinance
- Understanding of basic statistics (mean/average)

## Learning Objectives

By the end of this notebook, you will be able to:
1. Calculate and plot Simple Moving Averages (SMA)
2. Calculate and plot Exponential Moving Averages (EMA)
3. Identify market trends using moving averages
4. Recognize golden cross and death cross signals
5. Use moving averages as dynamic support and resistance
6. Generate trading signals based on moving average crossovers
7. Apply Malaysia-specific research findings on moving averages

## Introduction: The Most Popular Technical Indicator

**Moving averages are the foundation of technical analysis** - used by over 90% of traders worldwide.

### Why Moving Averages?

Moving averages help you:
- ✅ **Smooth out price noise**: See the trend beyond daily fluctuations
- ✅ **Identify trend direction**: Is the stock going up, down, or sideways?
- ✅ **Find entry/exit points**: Golden cross and death cross signals
- ✅ **Spot support/resistance**: Price often bounces from moving averages

### Malaysian Market Research

Research on **Bursa Malaysia** (10,000+ share positions) shows:
- **SMA 10 strategy**: 24% returns in 20 trading days, 35% in 26 days
- **Entry signal**: Price trades above SMA 10 with 2% filter
- **Best for**: Swing trading (3-15 day holding periods)

Let's master moving averages!

In [ ]:
# Setup: Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

# Visualization configuration
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 7)

np.random.seed(42)

print("✅ Environment setup complete!")

In [ ]:
# Download sample data for analysis
# We'll use Maybank (1155.KL) - highly liquid, suitable for learning

ticker = '1155.KL'
start_date = '2023-01-01'
end_date = '2024-12-31'

print(f"Downloading {ticker} (Maybank) data...\n")
data = yf.download(ticker, start=start_date, end=end_date, progress=False)

print(f"✅ Downloaded {len(data)} days of data")
print(f"Date range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
print(f"\nFirst few rows:")
data.head()

## 1. Simple Moving Average (SMA)

### What is SMA?

A **Simple Moving Average** is the average closing price over a specific number of periods.

### Formula

```
SMA = (P1 + P2 + P3 + ... + Pn) / n
```

Where:
- P = Price (usually Close or Adj Close)
- n = Number of periods

### Example (10-day SMA)

If the last 10 closing prices are: 10, 11, 10.5, 11.2, 11.5, 11, 11.3, 11.5, 11.8, 12

SMA 10 = (10 + 11 + 10.5 + 11.2 + 11.5 + 11 + 11.3 + 11.5 + 11.8 + 12) / 10 = **11.18**

### Common SMA Periods

| Period | Use Case | Typical Traders |
|--------|----------|----------------|
| **SMA 10** | Short-term trend | Day traders, swing traders |
| **SMA 20** | Short-medium trend | Swing traders |
| **SMA 50** | Medium-term trend | Position traders |
| **SMA 200** | Long-term trend | Investors, "trend followers" |

Let's calculate SMAs!

In [ ]:
# Calculate Simple Moving Averages
# pandas makes this incredibly easy with .rolling().mean()

# Calculate common SMA periods
data['SMA_10'] = data['Adj Close'].rolling(window=10).mean()
data['SMA_20'] = data['Adj Close'].rolling(window=20).mean()
data['SMA_50'] = data['Adj Close'].rolling(window=50).mean()
data['SMA_200'] = data['Adj Close'].rolling(window=200).mean()

print("Simple Moving Averages calculated!")
print("\nLatest values:")
print(f"Current Price: RM{data['Adj Close'].iloc[-1]:.2f}")
print(f"SMA 10:        RM{data['SMA_10'].iloc[-1]:.2f}")
print(f"SMA 20:        RM{data['SMA_20'].iloc[-1]:.2f}")
print(f"SMA 50:        RM{data['SMA_50'].iloc[-1]:.2f}")
print(f"SMA 200:       RM{data['SMA_200'].iloc[-1]:.2f}")

# Display sample with SMAs
print("\nSample data with SMAs:")
data[['Adj Close', 'SMA_10', 'SMA_20', 'SMA_50', 'SMA_200']].tail()

In [ ]:
# Visualize SMAs on price chart

plt.figure(figsize=(16, 8))

# Plot price
plt.plot(data.index, data['Adj Close'], linewidth=2, label='Price', color='black', alpha=0.7)

# Plot SMAs with different colors
plt.plot(data.index, data['SMA_10'], linewidth=1.5, label='SMA 10', color='#FF6B6B', alpha=0.8)
plt.plot(data.index, data['SMA_20'], linewidth=1.5, label='SMA 20', color='#4ECDC4', alpha=0.8)
plt.plot(data.index, data['SMA_50'], linewidth=2, label='SMA 50', color='#45B7D1', alpha=0.8)
plt.plot(data.index, data['SMA_200'], linewidth=2.5, label='SMA 200', color='#FFA07A', alpha=0.8)

plt.title('Maybank (1155.KL) with Simple Moving Averages', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (RM)', fontsize=12)
plt.legend(loc='best', fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("• Shorter SMAs (10, 20) follow price more closely")
print("• Longer SMAs (50, 200) are smoother and show long-term trend")
print("• Price crossing SMAs can signal trend changes")

## 2. Exponential Moving Average (EMA)

### What is EMA?

An **Exponential Moving Average** gives more weight to recent prices, making it more responsive to new information.

### Why EMA vs SMA?

| Feature | SMA | EMA |
|---------|-----|-----|
| **Calculation** | All prices equally weighted | Recent prices weighted more |
| **Responsiveness** | Slower to react | Faster to react |
| **Lag** | More lag | Less lag |
| **Best for** | Long-term trends | Short-term trends, day trading |

### Formula

```
EMA today = (Price today × Multiplier) + (EMA yesterday × (1 - Multiplier))

Multiplier = 2 / (n + 1)
```

### When to Use EMA?

- **Day trading**: EMA reacts faster to price changes
- **Volatile markets**: Better captures quick moves
- **Swing trading**: Many Malaysian traders prefer EMA for 3-15 day holds

Let's calculate and compare!

In [ ]:
# Calculate Exponential Moving Averages
# pandas uses .ewm() for exponential weighted moving average

data['EMA_10'] = data['Adj Close'].ewm(span=10, adjust=False).mean()
data['EMA_20'] = data['Adj Close'].ewm(span=20, adjust=False).mean()
data['EMA_50'] = data['Adj Close'].ewm(span=50, adjust=False).mean()

print("Exponential Moving Averages calculated!")
print("\nLatest values:")
print(f"Current Price: RM{data['Adj Close'].iloc[-1]:.2f}")
print(f"\nEMA vs SMA Comparison:")
print(f"EMA 10:  RM{data['EMA_10'].iloc[-1]:.2f}  |  SMA 10:  RM{data['SMA_10'].iloc[-1]:.2f}")
print(f"EMA 20:  RM{data['EMA_20'].iloc[-1]:.2f}  |  SMA 20:  RM{data['SMA_20'].iloc[-1]:.2f}")
print(f"EMA 50:  RM{data['EMA_50'].iloc[-1]:.2f}  |  SMA 50:  RM{data['SMA_50'].iloc[-1]:.2f}")

In [ ]:
# Compare SMA vs EMA side by side

fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# Top chart: SMA
axes[0].plot(data.index, data['Adj Close'], linewidth=1.5, label='Price', color='black', alpha=0.5)
axes[0].plot(data.index, data['SMA_10'], linewidth=2, label='SMA 10', color='#FF6B6B')
axes[0].plot(data.index, data['SMA_20'], linewidth=2, label='SMA 20', color='#4ECDC4')
axes[0].plot(data.index, data['SMA_50'], linewidth=2, label='SMA 50', color='#45B7D1')
axes[0].set_title('Simple Moving Averages (SMA)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Price (RM)', fontsize=11)
axes[0].legend(loc='best', fontsize=10)
axes[0].grid(True, alpha=0.3)

# Bottom chart: EMA
axes[1].plot(data.index, data['Adj Close'], linewidth=1.5, label='Price', color='black', alpha=0.5)
axes[1].plot(data.index, data['EMA_10'], linewidth=2, label='EMA 10', color='#FF6B6B', linestyle='--')
axes[1].plot(data.index, data['EMA_20'], linewidth=2, label='EMA 20', color='#4ECDC4', linestyle='--')
axes[1].plot(data.index, data['EMA_50'], linewidth=2, label='EMA 50', color='#45B7D1', linestyle='--')
axes[1].set_title('Exponential Moving Averages (EMA)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_ylabel('Price (RM)', fontsize=11)
axes[1].legend(loc='best', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Notice: EMAs (dashed lines) respond faster to price changes than SMAs")

## 3. Identifying Trends with Moving Averages

Moving averages help identify three market conditions:

### Trend Identification Rules

1. **Uptrend (Bullish)**:
   - Price > SMA 50 > SMA 200
   - Moving averages sloping upward
   - Short-term MAs above long-term MAs
   - **Action**: Look for buying opportunities

2. **Downtrend (Bearish)**:
   - Price < SMA 50 < SMA 200
   - Moving averages sloping downward
   - Short-term MAs below long-term MAs
   - **Action**: Stay out or look for shorting opportunities

3. **Sideways (Ranging)**:
   - Price oscillating around moving averages
   - Moving averages relatively flat
   - Frequent crossovers (whipsaw)
   - **Action**: Wait for clear trend or use range trading strategies

Let's create a trend identification system!

In [ ]:
# Trend identification function

def identify_trend(data, price_col='Adj Close', ma_short='SMA_50', ma_long='SMA_200'):
    """
    Identify market trend based on price and moving averages.
    
    Args:
        data (DataFrame): Stock data with moving averages
        price_col (str): Column name for price
        ma_short (str): Short-term moving average column
        ma_long (str): Long-term moving average column
    
    Returns:
        Series: Trend for each date ('Uptrend', 'Downtrend', 'Sideways')
    """
    trends = []
    
    for idx in range(len(data)):
        # Skip if moving averages not yet calculated
        if pd.isna(data[ma_long].iloc[idx]):
            trends.append('Unknown')
            continue
        
        price = data[price_col].iloc[idx]
        ma_s = data[ma_short].iloc[idx]
        ma_l = data[ma_long].iloc[idx]
        
        # Uptrend: Price > MA_short > MA_long
        if price > ma_s and ma_s > ma_l:
            trends.append('Uptrend')
        # Downtrend: Price < MA_short < MA_long
        elif price < ma_s and ma_s < ma_l:
            trends.append('Downtrend')
        # Sideways: Everything else
        else:
            trends.append('Sideways')
    
    return pd.Series(trends, index=data.index)

# Apply trend identification
data['Trend'] = identify_trend(data)

# Count trend occurrences
trend_counts = data['Trend'].value_counts()

print("Trend Analysis Results:")
print("=" * 50)
for trend, count in trend_counts.items():
    percentage = (count / len(data)) * 100
    print(f"{trend:12s}: {count:4d} days ({percentage:5.1f}%)")

print(f"\nCurrent trend: {data['Trend'].iloc[-1]}")

In [ ]:
# Visualize trends with color-coded background

plt.figure(figsize=(16, 8))

# Plot price and moving averages
plt.plot(data.index, data['Adj Close'], linewidth=2, label='Price', color='black', zorder=5)
plt.plot(data.index, data['SMA_50'], linewidth=2, label='SMA 50', color='blue', alpha=0.7, zorder=4)
plt.plot(data.index, data['SMA_200'], linewidth=2, label='SMA 200', color='red', alpha=0.7, zorder=4)

# Color-code background by trend
for i in range(1, len(data)):
    if data['Trend'].iloc[i] == 'Uptrend':
        plt.axvspan(data.index[i-1], data.index[i], alpha=0.1, color='green', zorder=1)
    elif data['Trend'].iloc[i] == 'Downtrend':
        plt.axvspan(data.index[i-1], data.index[i], alpha=0.1, color='red', zorder=1)
    else:
        plt.axvspan(data.index[i-1], data.index[i], alpha=0.05, color='gray', zorder=1)

plt.title('Maybank (1155.KL) - Trend Identification with Moving Averages', 
         fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (RM)', fontsize=12)
plt.legend(loc='best', fontsize=11)
plt.grid(True, alpha=0.3, zorder=2)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n💡 Background colors indicate trend:")
print("   🟢 Green = Uptrend (Price > SMA 50 > SMA 200)")
print("   🔴 Red = Downtrend (Price < SMA 50 < SMA 200)")
print("   ⚪ Gray = Sideways (mixed signals)")

## 4. Golden Cross and Death Cross

Two of the most famous trading signals in technical analysis!

### Golden Cross (Bullish Signal) 🌟

**Definition**: When a short-term MA crosses **above** a long-term MA

**Classic Example**: SMA 50 crosses above SMA 200

**Interpretation**: 
- Indicates potential start of uptrend
- Bullish momentum building
- Long-term buying opportunity

### Death Cross (Bearish Signal) ☠️

**Definition**: When a short-term MA crosses **below** a long-term MA

**Classic Example**: SMA 50 crosses below SMA 200

**Interpretation**:
- Indicates potential start of downtrend
- Bearish momentum building
- Exit signal for long positions

### Important Notes

⚠️ **Lagging Indicators**: Crosses occur AFTER trend has started

⚠️ **False Signals**: Can occur in sideways markets (whipsaws)

✅ **Best Used**: In trending markets, with confirmation from other indicators

Let's detect crossovers!

In [ ]:
# Detect Golden Cross and Death Cross signals

def detect_crossovers(data, ma_short='SMA_50', ma_long='SMA_200'):
    """
    Detect golden cross and death cross signals.
    
    Args:
        data (DataFrame): Stock data with moving averages
        ma_short (str): Short-term MA column
        ma_long (str): Long-term MA column
    
    Returns:
        DataFrame: Data with crossover signals
    """
    # Create a copy to avoid modifying original
    df = data.copy()
    
    # Calculate the difference between short and long MA
    df['MA_Diff'] = df[ma_short] - df[ma_long]
    
    # Detect crossovers
    # Golden Cross: MA_Diff crosses from negative to positive
    # Death Cross: MA_Diff crosses from positive to negative
    df['Signal'] = 'None'
    
    for i in range(1, len(df)):
        if pd.notna(df['MA_Diff'].iloc[i]) and pd.notna(df['MA_Diff'].iloc[i-1]):
            # Golden Cross
            if df['MA_Diff'].iloc[i-1] <= 0 and df['MA_Diff'].iloc[i] > 0:
                df.loc[df.index[i], 'Signal'] = 'Golden Cross'
            # Death Cross
            elif df['MA_Diff'].iloc[i-1] >= 0 and df['MA_Diff'].iloc[i] < 0:
                df.loc[df.index[i], 'Signal'] = 'Death Cross'
    
    return df

# Apply crossover detection
data = detect_crossovers(data, ma_short='SMA_50', ma_long='SMA_200')

# Find all crossovers
golden_crosses = data[data['Signal'] == 'Golden Cross']
death_crosses = data[data['Signal'] == 'Death Cross']

print("Crossover Signals Detected:")
print("=" * 60)
print(f"\n🌟 Golden Crosses (SMA 50 > SMA 200): {len(golden_crosses)}")
if len(golden_crosses) > 0:
    for date in golden_crosses.index:
        price = golden_crosses.loc[date, 'Adj Close']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f}")

print(f"\n☠️  Death Crosses (SMA 50 < SMA 200): {len(death_crosses)}")
if len(death_crosses) > 0:
    for date in death_crosses.index:
        price = death_crosses.loc[date, 'Adj Close']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f}")

In [ ]:
# Visualize crossovers on chart

plt.figure(figsize=(16, 9))

# Plot price and moving averages
plt.plot(data.index, data['Adj Close'], linewidth=1.5, label='Price', 
         color='black', alpha=0.6, zorder=3)
plt.plot(data.index, data['SMA_50'], linewidth=2.5, label='SMA 50', 
         color='#2E86AB', alpha=0.8, zorder=4)
plt.plot(data.index, data['SMA_200'], linewidth=2.5, label='SMA 200', 
         color='#A23B72', alpha=0.8, zorder=4)

# Mark Golden Crosses
if len(golden_crosses) > 0:
    plt.scatter(golden_crosses.index, golden_crosses['Adj Close'], 
               color='green', marker='^', s=300, label='Golden Cross', 
               zorder=5, edgecolors='darkgreen', linewidth=2)

# Mark Death Crosses
if len(death_crosses) > 0:
    plt.scatter(death_crosses.index, death_crosses['Adj Close'], 
               color='red', marker='v', s=300, label='Death Cross', 
               zorder=5, edgecolors='darkred', linewidth=2)

plt.title('Maybank (1155.KL) - Golden Cross & Death Cross Signals', 
         fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (RM)', fontsize=12)
plt.legend(loc='best', fontsize=11)
plt.grid(True, alpha=0.3, zorder=1)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n💡 Trading Strategy:")
print("   🌟 Golden Cross → Consider BUYING (uptrend likely starting)")
print("   ☠️  Death Cross → Consider SELLING/EXITING (downtrend likely starting)")
print("\n⚠️  Remember: These are LAGGING indicators. Trend has already started!")

## 5. Malaysian Market Strategy: SMA 10 with 2% Filter

Research on **10,000 Bursa Malaysia trades** shows this profitable strategy:

### The Strategy

**Entry Signal**: 
- Price trades **above** SMA 10 by at least **2%**
- Formula: `Price >= SMA_10 * 1.02`

**Exit Signal**:
- Price falls **below** SMA 10 by at least **2%**
- Formula: `Price <= SMA_10 * 0.98`

### Research Results

- **24% returns** in 20 trading days
- **35% returns** in 26 trading days
- Based on 10,000 share positions
- Works best in trending markets

### Why the 2% Filter?

The 2% filter prevents false signals from minor price fluctuations (reduces whipsaws).

Let's implement this Malaysian market-tested strategy!

In [ ]:
# Implement SMA 10 with 2% filter strategy

def sma10_filter_strategy(data, filter_pct=0.02):
    """
    Malaysian market SMA 10 strategy with percentage filter.
    
    Args:
        data (DataFrame): Stock data with SMA_10
        filter_pct (float): Filter percentage (0.02 = 2%)
    
    Returns:
        DataFrame: Data with trading signals
    """
    df = data.copy()
    
    # Calculate upper and lower bands (SMA 10 ± 2%)
    df['SMA10_Upper'] = df['SMA_10'] * (1 + filter_pct)
    df['SMA10_Lower'] = df['SMA_10'] * (1 - filter_pct)
    
    # Generate signals
    df['SMA10_Signal'] = 'Hold'
    df['Position'] = 0  # 1 = Long, 0 = No position
    
    position = 0
    
    for i in range(1, len(df)):
        if pd.notna(df['SMA_10'].iloc[i]):
            price = df['Adj Close'].iloc[i]
            
            # Entry: Price crosses above SMA 10 + 2%
            if position == 0 and price >= df['SMA10_Upper'].iloc[i]:
                df.loc[df.index[i], 'SMA10_Signal'] = 'BUY'
                position = 1
            
            # Exit: Price crosses below SMA 10 - 2%
            elif position == 1 and price <= df['SMA10_Lower'].iloc[i]:
                df.loc[df.index[i], 'SMA10_Signal'] = 'SELL'
                position = 0
            
            df.loc[df.index[i], 'Position'] = position
    
    return df

# Apply strategy
data = sma10_filter_strategy(data, filter_pct=0.02)

# Extract buy and sell signals
buy_signals = data[data['SMA10_Signal'] == 'BUY']
sell_signals = data[data['SMA10_Signal'] == 'SELL']

print("SMA 10 Strategy (2% Filter) Results:")
print("=" * 60)
print(f"\nBUY Signals: {len(buy_signals)}")
if len(buy_signals) > 0:
    print("\nRecent BUY signals:")
    for date in buy_signals.index[-5:]:  # Last 5
        price = buy_signals.loc[date, 'Adj Close']
        sma = buy_signals.loc[date, 'SMA_10']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f} (SMA10: RM{sma:.2f})")

print(f"\nSELL Signals: {len(sell_signals)}")
if len(sell_signals) > 0:
    print("\nRecent SELL signals:")
    for date in sell_signals.index[-5:]:  # Last 5
        price = sell_signals.loc[date, 'Adj Close']
        sma = sell_signals.loc[date, 'SMA_10']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f} (SMA10: RM{sma:.2f})")

In [ ]:
# Visualize SMA 10 strategy

plt.figure(figsize=(16, 9))

# Plot price
plt.plot(data.index, data['Adj Close'], linewidth=2, label='Price', 
         color='black', alpha=0.7, zorder=3)

# Plot SMA 10 and bands
plt.plot(data.index, data['SMA_10'], linewidth=2, label='SMA 10', 
         color='blue', alpha=0.8, zorder=4)
plt.plot(data.index, data['SMA10_Upper'], linewidth=1.5, label='SMA 10 + 2%', 
         color='green', linestyle='--', alpha=0.6, zorder=2)
plt.plot(data.index, data['SMA10_Lower'], linewidth=1.5, label='SMA 10 - 2%', 
         color='red', linestyle='--', alpha=0.6, zorder=2)

# Mark BUY signals
if len(buy_signals) > 0:
    plt.scatter(buy_signals.index, buy_signals['Adj Close'], 
               color='green', marker='^', s=200, label='BUY Signal', 
               zorder=5, edgecolors='darkgreen', linewidth=1.5)

# Mark SELL signals
if len(sell_signals) > 0:
    plt.scatter(sell_signals.index, sell_signals['Adj Close'], 
               color='red', marker='v', s=200, label='SELL Signal', 
               zorder=5, edgecolors='darkred', linewidth=1.5)

# Shade periods when in position
in_position = data[data['Position'] == 1]
if len(in_position) > 0:
    for i in range(len(in_position)):
        plt.axvspan(in_position.index[i], 
                   in_position.index[min(i+1, len(in_position)-1)], 
                   alpha=0.1, color='green', zorder=1)

plt.title('Maybank (1155.KL) - SMA 10 Strategy with 2% Filter\n(Malaysian Market Research-Based)', 
         fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (RM)', fontsize=12)
plt.legend(loc='best', fontsize=10, ncol=2)
plt.grid(True, alpha=0.3, zorder=0)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n💡 Strategy Rules:")
print("   ✅ BUY: When price crosses ABOVE SMA 10 + 2% (green arrow)")
print("   ❌ SELL: When price crosses BELOW SMA 10 - 2% (red arrow)")
print("   🟢 Green shading: Periods holding position")
print("\n📊 Research shows 24-35% returns in 20-26 trading days!")

## 6. Practice Exercises

Apply what you've learned about moving averages!

### Exercise 1: Compare SMA vs EMA Performance

Download data for **Public Bank (1295.KL)** and:
1. Calculate both SMA 20 and EMA 20
2. Create a comparison chart
3. Which one responds faster to price changes?
4. During which market conditions (trending vs sideways) does each perform better?

In [ ]:
# YOUR CODE HERE



### Exercise 2: Multi-Stock Crossover Finder

Create a function that scans multiple Malaysian stocks for recent golden crosses or death crosses. Test it on:
- Banking sector: 1155.KL, 1295.KL, 1023.KL
- Construction: 5398.KL, 5211.KL

Which stocks have bullish signals (golden cross) in the last 30 days?

In [ ]:
# YOUR CODE HERE



### Exercise 3: Optimize the Filter Percentage

The Malaysian research used a 2% filter. Test different filter percentages (1%, 2%, 3%, 5%) on Gamuda (5398.KL) data:
1. Count number of trades generated
2. Calculate which filter produces the best signal quality (fewer whipsaws)
3. Which filter percentage would you choose and why?

In [ ]:
# YOUR CODE HERE



### Exercise 4: Multiple Timeframe Analysis

Download **Sime Darby Plantation (5285.KL)** and analyze trends using three timeframes:
1. Short-term: SMA 10 and SMA 20
2. Medium-term: SMA 50
3. Long-term: SMA 200

Are all three timeframes aligned (all bullish or all bearish)? If not, which timeframe would you trust more for your trading decision?

In [ ]:
# YOUR CODE HERE



## 7. Summary and Key Takeaways

Excellent work! You've mastered moving averages - the foundation of technical analysis.

### ✅ Skills Mastered

1. **Simple Moving Average (SMA)**: Calculate and interpret trend smoothing
2. **Exponential Moving Average (EMA)**: Faster-reacting alternative to SMA
3. **Trend Identification**: Recognize uptrends, downtrends, and sideways markets
4. **Golden Cross/Death Cross**: Major trend reversal signals
5. **Malaysian Strategy**: SMA 10 with 2% filter (research-proven)
6. **Multi-Timeframe Analysis**: Use multiple MAs for comprehensive view

### 📊 Key Concepts

**Moving Average Periods**:
- SMA 10, 20: Short-term (swing trading)
- SMA 50: Medium-term (position trading)
- SMA 200: Long-term (investing)

**Trend Rules**:
- **Uptrend**: Price > SMA 50 > SMA 200
- **Downtrend**: Price < SMA 50 < SMA 200
- **Sideways**: Mixed signals, flat MAs

**Crossover Signals**:
- **Golden Cross**: Short MA crosses above long MA (bullish)
- **Death Cross**: Short MA crosses below long MA (bearish)

### 🇲🇾 Malaysian Market Insights

Research on **10,000 Bursa Malaysia trades**:
- **SMA 10 + 2% filter**: 24-35% returns in 20-26 days
- **Best for**: Swing trading (3-15 day holds)
- **Key**: Filter reduces false signals (whipsaws)

### ⚠️ Important Warnings

1. **Lagging Indicators**: MAs show trend AFTER it starts
2. **Whipsaws**: False signals in sideways markets
3. **Not Standalone**: Use with other indicators (RSI, MACD, volume)
4. **Transaction Costs**: Frequent signals = higher costs

### 🎯 What's Next?

In **Module 04: RSI and MACD Indicators**, you'll learn:
- Relative Strength Index (RSI) for overbought/oversold
- MACD for momentum and crossovers
- Combining RSI + MACD with moving averages
- Malaysian research: RSI + MACD combinations (73% win rate!)

### 💡 Pro Tips

1. **Use multiple timeframes**: Align short and long-term trends
2. **Filter for quality**: 2% filter reduces noise
3. **Confirm with volume**: High volume confirms breakouts
4. **Trending markets**: MAs work best in trends, not ranges
5. **Paper trade first**: Test strategies before real money

### 📚 Additional Resources

- Malaysian research paper: "Swing Trading Strategy Based on SMA 10"
- Book: *Technical Analysis of the Financial Markets* by John Murphy
- i3investor forums: Real trader discussions on MA strategies

---

**Congratulations on completing Module 03!** 🎉

You now understand the most widely-used technical indicator. Moving averages will be the foundation for everything you learn next.

**Next up**: `04_rsi_and_macd_indicators.ipynb` - Learn momentum indicators!

---

*"The trend is your friend." - Trading Proverb*